In [1]:
import os
import openai
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from dotenv import load_dotenv

load_dotenv() # Load variables from .env file



False

In [2]:
# load document
api_key = os.environ.get("OPENAI_API_KEY") 
loader = PyPDFLoader("./DMV_Handbook.pdf")
documents = loader.load()

# split the documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# select which embeddings we want to use
embeddings = OpenAIEmbeddings(openai_api_key = api_key)

# create the vectorestore to use as the index
db = Chroma.from_documents(texts, embeddings)


Using embedded DuckDB without persistence: data will be transient


In [3]:
#Credits : Inspired by Kimchoi, https://github.com/kimchoi-jjiggae/quizmegpt

def generate_questions(doc):
    # Construct the prompt for the GPT model
    prompt = f"Generate 1 factual test question whose answer should be a single word, based on this text, and do not include the answer: {doc}"  # noqa: E501
    
    # Call the OpenAI API for chat completion
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a teacher. I need you to help write me exam questions.",  # noqa: E501
            },
            {"role": "user", "content": prompt},
        ],
        max_tokens=1200,
        n=1,
        stop=None,
        temperature=0.7,
    )

    # Extract the generated HTML from the response
    reply = response["choices"][0]["message"]["content"]
    return reply


def evaluate_questions(query, result, user_answer):
    # Construct the prompt for the GPT model
    prompt = f"give encouraging feedback to this student, telling them if they are correct or wrong and how to improve if applicable: {query} 'correct answer': ' {result['result']}', student answer: {user_answer}"  # noqa: E501

    # Call the OpenAI API for chat completion
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a teacher. I need you to help grade exams.",  # noqa: E501
            },
            {"role": "user", "content": prompt},
        ],
        max_tokens=1200,
        n=1,
        stop=None,
        temperature=0.7,
    )

    # Extract the generated HTML from the response
    reply = response["choices"][0]["message"]["content"]
    return reply


def generate_followup_questions(doc, query, result, user1_answer, user2_answer):
    # Construct the prompt for the GPT model
    prompt = f"Here is the question and student answers. If both students answered correctly, please suggest another question. If one student got it correct but the other didn't please generate a follow up question so that one peer can teach another peer, ask another factual test question whose answer should be a single word based on this text. If both got it wrong, ask the same question again. 'Document': ' {doc} ', 'question': ' {query} ', 'correct answer': ' {result['result']}', student 1 answer: {user1_answer}, student 2 answer: {user2_answer}"  # noqa: E501
    
    # Call the OpenAI API for chat completion
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a teacher. I need you to create a collaborative exercise so peers can learn from each other.",  # noqa: E501
            },
            {"role": "user", "content": prompt},
        ],
        max_tokens=1200,
        n=1,
        stop=None,
        temperature=0.7,
    )

    # Extract the generated HTML from the response
    reply = response["choices"][0]["message"]["content"]
    return reply



In [4]:
# expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})

# create a chain to answer questions 
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=True)
# print("TEXT", texts[5])
query = generate_questions(texts[20])

print ('********************\n\n')
print('GENERATED QUESTION:', query, '\n\n')
result = qa({"query": query})
print ('********************\n')
user1_answer = input("Enter your answer student 1: ")
user2_answer = input("Enter your answer student 2: ")

print ('\n********* Generating Feedback ***********\n')
feedback1 = evaluate_questions(query, result, user1_answer)
print('FEEDBACK Student 1:', feedback1, '\n\n')
feedback2 = evaluate_questions(query, result, user2_answer)
print('FEEDBACK Student 2:', feedback2, '\n\n')

print ('********* Complete Answer ***********\n')  
print('CORRECT ANSWER:', result['result'])

print ('\n********* Generating Follow-up collaboration ***********\n')
followup = generate_followup_questions(texts[20], query, result, user1_answer, user2_answer)
print('GENERATED Followup:', followup, '\n\n')

print ('\n\n\n********* Keep Going! ***********\n')  


# print ('********************\n')
# user1_answer = input("Enter your answer student 1: ")
# user2_answer = input("Enter your answer student 2: ")


# print ('\n********* Generating Feedback ***********\n')
# feedback1 = evaluate_questions(query, result, user1_answer)
# print('FEEDBACK Student 1:', feedback1, '\n\n')
# feedback2 = evaluate_questions(query, result, user2_answer)
# print('FEEDBACK Student 2:', feedback2, '\n\n')


# print ('\n********* Generating Follow-up collaboration ***********\n')
# followup = generate_followup_questions(texts[20], query, result, user1_answer, user2_answer)
# print('GENERATED Followup:', followup, '\n\n')

# print ('\n\n\n********* Keep Going! ***********\n')  


********************


GENERATED QUESTION: What does a single solid yellow line on the center of a road with two-way traffic indicate? 


********************


********* Generating Feedback ***********

FEEDBACK Student 1: Dear student, thank you for submitting your answer. Unfortunately, your answer does not address the question. The correct answer is: "A single solid yellow line indicates that you should not pass a vehicle in front of you." It is important to pay attention to road markings as they provide valuable information for safe driving. Keep practicing and paying attention to details. You'll get there! 


FEEDBACK Student 2: Dear student, thank you for taking the time to answer this question. Unfortunately, your response 'fkshfs' is not the correct answer. The correct answer is 'A single solid yellow line indicates that you should not pass a vehicle in front of you.' Please take some time to review this information and remember to read questions carefully before answering. Ke